In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor, plot_tree
import cvxopt
import patsy
import os

np.set_printoptions(precision=2, suppress=True)
pl.Config.set_float_precision(2)

polars.config.Config

In [2]:
# importing the oj dataset
os.chdir('/Users/noaht/OneDrive/Desktop/ECON_487')
data_oj = pl.read_csv('oj.csv')

In [ ]:
data_oj = (
    data_oj
    # changing the minute maid alias so there's less syntax problems in the future 
    .with_columns(
        pl.when(pl.col("brand") == "minute.maid")
        .then(pl.lit("minute_maid"))
        .otherwise(pl.col("brand"))
        .alias("brand")  
))

In [33]:
pd_oj = data_oj.to_pandas()

In [4]:
# adding a log_price column
data_oj = data_oj.with_columns(
    pl.col('price').log().alias('log_price')
)

In [5]:
# adding a row id
data_oj = data_oj.with_row_index('row_id')

In [6]:
# lag stuff
# table to join
df1 = (
    data_oj
    .select(['store', 'brand', 'week', 'log_price'])
    .sort(['store', 'brand', 'week'])
    .with_columns(
        (pl.col('week')+1).alias('week'),
    )
)
# dataframe with lagged log price
df2 = (
    data_oj
    .join(df1, on=['brand','store','week'], how='left')
)
df2 = df2.sort(["store", "brand", "week"])

In [19]:
# refining stuff
lag_price_data = (
    df2
    .rename({'log_price_right': 'lag_price'})
    # taking out the null values
    .filter(pl.col("lag_price").is_not_null())
    # changing the minute maid alias so there's less syntax problems in the future 
    .with_columns(
        pl.when(pl.col("brand") == "minute.maid")
        .then(pl.lit("minute_maid"))
        .otherwise(pl.col("brand"))
        .alias("brand")  
))

In [8]:
pd_lpd = lag_price_data.to_pandas()

cross validation

In [9]:
folds = KFold(n_splits=5, shuffle=True, random_state=487)

In [10]:
absurd_formula = 'logmove ~ (log_price + lag_price + +feat + AGE60 + EDUC + ETHNIC + C(brand) +' \
          'INCOME + HHLARGE + WORKWOM + HVAL150 + SSTRDIST + SSTRVOL + ' \
          'CPDIST5 + CPWVOL5)**2-1'

In [11]:
alphas = np.logspace(-3, -2, 5)  

# alpha --> penalty 
alpha_results = []

for alpha in alphas:
    mse = []
    
    for train_i, test_i in folds.split(lag_price_data):
        train_data = lag_price_data[train_i].to_pandas()
        test_data = lag_price_data[test_i].to_pandas()
        
        y_train, X_train = patsy.dmatrices(absurd_formula, data=train_data, return_type='dataframe')
        y_test, X_test = patsy.dmatrices(absurd_formula, data=test_data, return_type='dataframe')
        
        model = sm.OLS(y_train, X_train).fit_regularized(method = 'elastic_net', L1_wt=1, alpha=alpha)
        
        y_hat = model.predict(X_test)
        
        model_mse = mean_squared_error(y_test, y_hat)
        mse.append(model_mse)
    
    avg_mse = np.mean(mse)
    
    alpha_results.append({
        'alpha': alpha,
        'avg_mse': avg_mse})
    print(f"Alpha = {alpha:.4f}; Avg MSE = {avg_mse:.2f}")

results_df = pd.DataFrame(alpha_results)
best_alpha = results_df.loc[results_df['avg_mse'].idxmin(), 'alpha']
print(f"\nBest alpha: {best_alpha:.4f}")

Alpha = 0.0010; Avg MSE = 0.39
Alpha = 0.0018; Avg MSE = 0.39
Alpha = 0.0032; Avg MSE = 0.40
Alpha = 0.0056; Avg MSE = 0.42
Alpha = 0.0100; Avg MSE = 0.44

Best alpha: 0.0010


In [12]:
""" 
convert the absurd_formula variables into a numerical matrix the regression model can use
    this formula does different things
    creates dummy variables for the categorical variables
"""
y, X = patsy.dmatrices(absurd_formula, data=pd_lpd, return_type='dataframe')

# LASSO Regression 
best_model = sm.OLS(y, X).fit_regularized(method = 'elastic_net', L1_wt=1, alpha=.001)

In [13]:
""" 
return the non-zero parameters after LASSO is performed
    this is using the absurd formula which probably wouldn't be done in practice
    the absurd formula is just used to illustrate a point
""" 
# create a series mapping coefficient names to thier values
coef = pd.Series(best_model.params, index=X.columns)

# find the non-zero coefficients --> survived LASSO
nonzero_cols = np.abs(coef) != 0 

# maka a polars dataframe of the features that survived
nonzero_features = pl.DataFrame({
    'feature': pl.Series(X.columns[nonzero_cols])
})

with pl.Config(tbl_rows=200):
    print(nonzero_features)

shape: (61, 1)
┌─────────────────────────────────┐
│ feature                         │
│ ---                             │
│ str                             │
╞═════════════════════════════════╡
│ C(brand)[dominicks]             │
│ C(brand)[minute_maid]           │
│ C(brand)[tropicana]             │
│ log_price                       │
│ log_price:C(brand)[T.minute_ma… │
│ log_price:C(brand)[T.tropicana… │
│ lag_price                       │
│ lag_price:C(brand)[T.minute_ma… │
│ feat                            │
│ feat:C(brand)[T.tropicana]      │
│ EDUC                            │
│ EDUC:C(brand)[T.tropicana]      │
│ INCOME                          │
│ C(brand)[T.minute_maid]:INCOME  │
│ C(brand)[T.tropicana]:INCOME    │
│ C(brand)[T.minute_maid]:HVAL15… │
│ C(brand)[T.tropicana]:HVAL150   │
│ SSTRDIST                        │
│ C(brand)[T.tropicana]:SSTRDIST  │
│ SSTRVOL                         │
│ C(brand)[T.tropicana]:SSTRVOL   │
│ CPDIST5                         │
│ C(brand)[T.

In [14]:
# refit the model only using the the variables that LASSO selects
refit_lasso = sm.OLS(y, X).fit_regularized(method = 'elastic_net', L1_wt=1, alpha=.001, refit=True)

In [15]:
# show all the variables that affect log_price
log_price_coefs = coef[coef.index.str.contains("log_price", case=False)]
print(log_price_coefs)

log_price                           -1.617964
log_price:C(brand)[T.minute_maid]    0.247364
log_price:C(brand)[T.tropicana]      0.859430
log_price:lag_price                 -1.466326
log_price:feat                      -0.628343
log_price:AGE60                      0.000000
log_price:EDUC                       0.000000
log_price:ETHNIC                     0.000000
log_price:INCOME                    -0.025758
log_price:HHLARGE                    0.000000
log_price:WORKWOM                    0.000000
log_price:HVAL150                    0.319606
log_price:SSTRDIST                  -0.021176
log_price:SSTRVOL                    0.000000
log_price:CPDIST5                    0.036387
log_price:CPWVOL5                    0.000000
dtype: float64


# Question 3

In [21]:
# wide data --> use for elasticities
wide_data = (
    lag_price_data
    .select(["store", "week", "brand", "log_price", 'lag_price',"feat"])
    .unpivot(index=["store", "week", "brand", "feat"], on=["log_price", 'lag_price', 'feat'])
    # naming stuff
    .with_columns(
        pl.concat_str([pl.col("variable"), pl.lit("_"), pl.col("brand")]).alias("name")
    )
    .drop("brand")
    .pivot(index=["store", "week"], on="name", values="value")
)

# table we're going to use for cross price elasticities
cross_price_data = (
    lag_price_data
    .select(['store', 'week', 'logmove', 'brand'])
    .join(wide_data, how='left', on=['store', 'week'])
)

In [27]:
# convert polars dataframe to pandas
cp_pd = cross_price_data.to_pandas()

def fit_brand(group):
    fit = smf.ols(
        "logmove ~ log_price_dominicks + log_price_minute_maid + log_price_tropicana",
        data=group
    ).fit()
    return {
        # get the brand name
        "q_var": group["brand"].iloc[0],
        # extract cross price effects (coefficients)
        "log_price_dominicks": fit.params["log_price_dominicks"],
        "log_price_minute_maid": fit.params["log_price_minute_maid"],
        "log_price_tropicana": fit.params["log_price_tropicana"]
    }
# loop through each brand --> list of dictionaries
results = [fit_brand(g) for _, g in cp_pd.groupby("brand")]
# turn into clean table
out = pl.DataFrame(results).sort("q_var")
out

q_var,log_price_dominicks,log_price_minute_maid,log_price_tropicana
str,f64,f64,f64
"""dominicks""",-3.56,1.25,0.06
"""minute_maid""",0.91,-3.86,1.18
"""tropicana""",0.27,0.43,-2.87


### interact all price variables with feat

In [29]:
def price_feat_reg(brand_var: str, cross_price_data: pl.DataFrame) -> pl.DataFrame:
    
    sub = cross_price_data.filter(pl.col("brand") == brand_var).to_pandas()
    
    feat_col = f"feat_{brand_var}"
    formula = (
        "logmove ~ (log_price_dominicks + log_price_minute_maid + log_price_tropicana) "
        f"* {feat_col}"
    )
    fit = smf.ols(formula, data=sub).fit()

    coefs = fit.params
    return pl.DataFrame({
        "q_var": [brand_var],
        "log_price_dominicks": [coefs['log_price_dominicks']],
        "log_price_minute_maid": [coefs["log_price_minute_maid"]],
        "log_price_tropicana": [coefs["log_price_tropicana"]],
    })


brand_list = lag_price_data.select("brand").unique().to_series().to_list()
rows = [price_feat_reg(b, cross_price_data) for b in brand_list]
out = pl.concat(rows, how="vertical").sort("q_var")
out

q_var,log_price_dominicks,log_price_minute_maid,log_price_tropicana
str,f64,f64,f64
"""dominicks""",-2.87,0.81,-0.26
"""minute_maid""",0.56,-2.35,0.33
"""tropicana""",0.15,0.28,-2.17


it's interesting that the cross price elasticity between dominicks and tropicana is negative. 

## I have to check this again

In [ ]:
# it looks like minute maid has a pretty big effect

feat_col = "feat_minute_maid"
formula = "logmove ~ (log_price_dominicks + log_price_minute_maid + log_price_tropicana)*feat_minute_maid"

fit_dominicks = smf.ols(formula, data=cross_price_data.filter(pl.col('brand')=='dominicks')).fit()
fit_minute_maid = smf.ols(formula, data=cross_price_data.filter(pl.col('brand')=='minute_maid')).fit()
fit_tropicana = smf.ols(formula, data=cross_price_data.filter(pl.col('brand')=='tropicana')).fit()

print(fit_dominicks.summary())
# print(fit_minute_maid.summary())
print(fit_tropicana.summary())

                            OLS Regression Results                            
Dep. Variable:                logmove   R-squared:                       0.444
Model:                            OLS   Adj. R-squared:                  0.444
Method:                 Least Squares   F-statistic:                     1064.
Date:                Sat, 31 Jan 2026   Prob (F-statistic):               0.00
Time:                        22:20:59   Log-Likelihood:                -12180.
No. Observations:                9336   AIC:                         2.438e+04
Df Residuals:                    9328   BIC:                         2.443e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [38]:
# tree data
oj_tree = (
    lag_price_data
    .with_columns(q = pl.col("logmove").exp())
    .with_columns(
        # weighted by quantity
        weighted_price = (
            (pl.col("price") * pl.col("q")).sum() / pl.col("q").sum()
        ).over(["store", "week"])
    )
)

oj_tree

row_id,store,brand,week,logmove,feat,price,AGE60,EDUC,ETHNIC,INCOME,HHLARGE,WORKWOM,HVAL150,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5,log_price,lag_price,q,weighted_price
u32,i64,str,i64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
222,2,"""dominicks""",47,8.83,1,2.09,0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,0.74,0.99,6848.00,2.51
223,2,"""dominicks""",48,7.97,0,2.09,0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,0.74,0.74,2880.00,3.35
225,2,"""dominicks""",51,10.14,1,1.89,0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,0.64,0.74,25344.00,2.43
226,2,"""dominicks""",52,9.28,0,1.89,0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,0.64,0.64,10752.00,2.66
227,2,"""dominicks""",53,8.80,0,1.89,0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,0.64,0.64,6656.00,2.35
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
28746,137,"""tropicana""",156,11.31,1,2.49,0.21,0.53,0.11,10.97,0.09,0.33,0.86,6.03,0.71,0.77,0.33,0.91,1.02,81920.00,2.36
28747,137,"""tropicana""",157,10.63,0,2.66,0.21,0.53,0.11,10.97,0.09,0.33,0.86,6.03,0.71,0.77,0.33,0.98,0.91,41280.00,2.25
28748,137,"""tropicana""",158,9.94,0,3.11,0.21,0.53,0.11,10.97,0.09,0.33,0.86,6.03,0.71,0.77,0.33,1.13,0.98,20672.00,2.60


# Question 5

In [39]:

# why do I need this?
oj_cols = oj_tree.columns

reg_tree_data = oj_tree.select(pl.exclude('row_id','logmove','store','brand','week','lag_week','log_price','feat','price','q','lag_price'))

reg_tree_data

AGE60,EDUC,ETHNIC,INCOME,HHLARGE,WORKWOM,HVAL150,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5,weighted_price
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,2.51
0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,3.35
0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,2.43
0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,2.66
0.23,0.25,0.11,10.55,0.10,0.30,0.46,2.11,1.14,1.93,0.38,2.35
…,…,…,…,…,…,…,…,…,…,…,…
0.21,0.53,0.11,10.97,0.09,0.33,0.86,6.03,0.71,0.77,0.33,2.36
0.21,0.53,0.11,10.97,0.09,0.33,0.86,6.03,0.71,0.77,0.33,2.25
0.21,0.53,0.11,10.97,0.09,0.33,0.86,6.03,0.71,0.77,0.33,2.60


In [ ]:

y = reg_tree_data.select('weighted_price').to_numpy().ravel()
X = reg_tree_data.select(pl.exclude('weighted_price')).to_numpy()

# Regression Tree
tree = DecisionTreeRegressor(ccp_alpha=0.002, random_state=487)
tree.fit(X, y)

,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,487
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.002
